In [1]:
# load data set, the data set contains the works of shakespere
#!python -m wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt


Saved under input (1).txt


In [1]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
print("total length of dataset:", len(text))

total length of dataset: 1115394


In [3]:
#print few characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
# collect the distinct charset
chars = sorted(set(text))
vocab_size = len(chars)
print("total distinct chars:", ''.join(chars))
print("size of vocabulary:", vocab_size)
# note /n is also a characters in our vocab

total distinct chars: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
size of vocabulary: 65


In [5]:
# embedding the chars

# create maps for embeddingss
char_to_value = { ch:i for i,ch in enumerate(chars)}
value_to_char = { i:ch for i,ch in enumerate(chars)}

# create lambda function to encode and decode the given text
encode = lambda s: [char_to_value[ch] for ch in s]
decode = lambda s: [value_to_char[vl] for vl in s]

print(encode("shakespere"))
print(decode(encode("shakespere")))

[57, 46, 39, 49, 43, 57, 54, 43, 56, 43]
['s', 'h', 'a', 'k', 'e', 's', 'p', 'e', 'r', 'e']


In [6]:
#create a data tensor
import torch
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [7]:
# split the data into train and validation sets (0.9:0.1)
percentile = int(0.9 * len(data))
train_data = data[:percentile]
val_data = data[percentile:]
print(train_data.shape, val_data.shape)

torch.Size([1003854]) torch.Size([111540])


In [8]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [9]:
# the ouput would be the next word for the set of input 
x = train_data[:block_size]
y = train_data[1:block_size+1]
for i in range(block_size):
    x_itr = x[:i+1]
    y_itr = y[i]
    print(f"for the input {x_itr} output is {y_itr}")

for the input tensor([18]) output is 47
for the input tensor([18, 47]) output is 56
for the input tensor([18, 47, 56]) output is 57
for the input tensor([18, 47, 56, 57]) output is 58
for the input tensor([18, 47, 56, 57, 58]) output is 1
for the input tensor([18, 47, 56, 57, 58,  1]) output is 15
for the input tensor([18, 47, 56, 57, 58,  1, 15]) output is 47
for the input tensor([18, 47, 56, 57, 58,  1, 15, 47]) output is 58


In [10]:
# we write code to get the batches of inputs
batch_size = 4 # no of sequences the model process in parallel
block_size = 8 # max context length for the prediction
# add and seed value to get same random numbers everytime
torch.manual_seed(1337)

def get_batch(split):
    # generate appropriate data
    data = train_data if split == "train" else val_data
    offsets = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in offsets])
    y = torch.stack([data[i+1:i+block_size+1] for i in offsets])
    return x,y

xb, yb = get_batch("train")
print(f"input: {xb}")
print(f"input size: {xb.shape}")
print(f"output: {yb}")
print(f"output size: {yb.shape}")

input: tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
input size: torch.Size([4, 8])
output: tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
output size: torch.Size([4, 8])


In [11]:
# implent BigramLanguageModel to embedd the data
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        # initialize a embedding vector
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets = None):
        # idx and targets are both of size (B(batch), T(block))
        logits = self.token_embedding_table(idx) # (B, T, C)
        
        if targets is None:
            loss = None
        else:
            # define the loss we use the cross entropy function
            # but the cross_entropy expects the 2nd dimension to be C
            # hence we reshape the arrays
            B, T, C = logits.shape
        
            # view function reshape the tensor
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices of current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus on the last timestamp
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            # dim=-1 reperesents to apply softmax to last dimension
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

blm = BigramLanguageModel(vocab_size)
logits, loss = blm(xb, yb)
print(logits.shape)
print(loss)

# test the model initally
print(''.join(decode(blm.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist())))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [12]:
# optimizer
optimizer = torch.optim.AdamW(blm.parameters(), lr = 1e-3)

In [15]:
# hyperparameters
batch_size = 32
block_size = 8
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
eval_iters = 200

In [32]:
for steps in range(10000):
    xb, yb = get_batch('train')
    logits, loss = blm(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
    if(steps+1)%500 == 0:
        print(f'completed {steps+1}/10000, loss={loss.item():.4f}')
    
print(loss.item())

completed 500/10000, loss=2.4568
completed 1000/10000, loss=2.3789
completed 1500/10000, loss=2.4362
completed 2000/10000, loss=2.3088
completed 2500/10000, loss=2.4568
completed 3000/10000, loss=2.5789
completed 3500/10000, loss=2.3757
completed 4000/10000, loss=2.5029
completed 4500/10000, loss=2.5389
completed 5000/10000, loss=2.4516
completed 5500/10000, loss=2.4302
completed 6000/10000, loss=2.5638
completed 6500/10000, loss=2.4534
completed 7000/10000, loss=2.4199
completed 7500/10000, loss=2.4469
completed 8000/10000, loss=2.4801
completed 8500/10000, loss=2.4538
completed 9000/10000, loss=2.4580
completed 9500/10000, loss=2.2790
completed 10000/10000, loss=2.4072
2.4072139263153076


In [14]:
print(''.join(decode(blm.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=200)[0].tolist())))


lso br. ave aviasurf my, yxMPZI ivee iuedrd whar ksth y h bora s be hese, woweee; the! KI 'de, ulseecherd d o blllando;LUCEO, oraingofof win!
RIfans picspeserer hee tha,
TOFonk? me ain ckntoty ded. bo


In [ ]:
# the above was the simplest model made but it was only taking the 
# last generated token as the input, lets build a model which
# takes previous token to build the context

In [16]:
# @torch.no_grad()
# def estimate_loss():
#     out = {}
#     # set the model into eval mode
#     model.eval()
#     for split in ['train', 'val']:
#         losses = torch.zeros(eval_iters)
#         for k in rangel(eval_iters):
#             X, Y = get_batch(split)
#             logits, loss = model(X, Y)
#             losses[k] = loss.item()
#         out[split] = losses.mean()
#     # set the model back into train mode
#     model.train()
#     return out

In [20]:
# for iter in range(max_iters):
    
#     if iter % eval_interval == 0:
#         losses = estimate_loss()
#         print(f"step {iter}: trian loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    
#     xb, yb = get_batch('train')
    
#     logits, loss = blm(xb, yb)
#     optimizer.zero_grad(set_to_none=True)
#     loss.backward()
#     optimizer.step()
    
# #generate from the model
# cnotext = torch.zeros((1, 1), dtype = torch.long)
# print(decode(blm.generate(context, max_new_tokens=500)[0].tolist()))

In [23]:
# mathematical trick in self-attention
# we want in our character in self attention model to communicate 
# with only the previous characters hence we use the following trick

B, T, C = 4,8,2

# we want x[b, t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B, T, C))
x = torch.randn(B, T, C)
print(x.shape)

#verison 1
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] #(t,C)
        xbow[b, t] = torch.mean(xprev, 0)

torch.Size([4, 8, 2])


In [29]:
#version 2

# here we multiply our matrix with the tril matrix which is the lower
# traingular matrix and it ensures the character is only loooking 
# the previous characters
wei = torch.tril(torch.ones(T, T)) # gives you a lower triangular matrix
wei = wei/wei.sum(1, keepdim=True) # gives you the average of the row
xbow2 = wei @ x #(T, T) @ (B, T, C) --> (B, T, C)
# when you multiply above wei with x as wei is 2 dimensional
# python broadcasts it to (B, T, T) dimension

# checks if the two matrices are identical with the margin of error
torch.allclose(xbow, xbow2)

True

In [31]:
# version 3: use softmax

# softmax can be replaced with the division with the sum as the softmax
# function does the same thing
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf')) # filles 0 with -inf
wei = F.softmax(wei, dim = -1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [36]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [43]:
# version 4: self-attention
torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

# lets see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias = False) 
query = nn.Linear(C, head_size, bias = False)
value = nn.Linear(C, head_size, bias = False)
k = key(x) #(B, T, 16)
q = query(x) #(B, T, 16)
v = value(x) #(B, T, 16)
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) --> (B, T, T)

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim = -1)

out = wei @ v

out.shape

torch.Size([4, 8, 16])

In [44]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [45]:
# we divide the wei matrix with sqrt(headsize) to normalize the variance

wei = q @ k.transpose(-2, -1) * head_size ** -0.5

In [47]:
wei.var()

tensor(0.1201, grad_fn=<VarBackward0>)